# <center>National Anthem Clustering</center>

## Modules

In [49]:
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.cm as cm
import matplotlib.pyplot as plt


from sklearn import cluster
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_samples, silhouette_score

import re
import nltk
import nltk.corpus
from wordcloud import WordCloud
from nltk import SnowballStemmer
from nltk.tokenize import word_tokenize

import folium
import warnings
from unidecode import unidecode
from branca.element import Figure



%matplotlib inline
nltk.download('punkt')
nltk.download('stopwords')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\samra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Initialize dataframe

In [50]:
df = pd.read_csv('./anthems.csv', encoding='utf-8')
df.columns = map(str.lower, df.columns)
df.head()

,country,alpha-2,alpha-3,continent,anthem
0,Albania,AL,ALB,Europe,"Around our flag we stand united, With one wish..."
1,Armenia,AM,ARM,Europe,"Our Fatherland, free, independent, That has fo..."
2,Austria,AT,AUT,Europe,"Land of mountains, land by the river, Land of ..."
3,Azerbaijan,AZ,AZE,Europe,"Azerbaijan, Azerbaijan! The glorious Fatherlan..."
4,Belarus,BY,BLR,Europe,"We, Belarusians, are peaceful people, Wholehea..."


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   country    190 non-null    object
 1   alpha-2    189 non-null    object
 2   alpha-3    190 non-null    object
 3   continent  190 non-null    object
 4   anthem     190 non-null    object
dtypes: object(5)
memory usage: 7.6+ KB


In [52]:
df.describe()

,country,alpha-2,alpha-3,continent,anthem
count,190,189,190,190,190
unique,190,188,190,6,188
top,Albania,CH,ALB,Africa,"Arise, ye who refuse to be slaves; With our ve..."
freq,1,2,1,56,2


In [53]:
df.isnull().sum()

country      0
alpha-2      1
alpha-3      0
continent    0
anthem       0
dtype: int64

In [54]:
df.loc[df['alpha-2'].isnull()]

,country,alpha-2,alpha-3,continent,anthem
168,Namibia,NaN,NAM,Africa,Namibia land of the brave Freedom fight we he ...


Now, setting the alpha-2 country code of Namibia to 'NA'.

In [63]:
df.loc[df['country'] == 'Namibia', ['alpha-2']] = 'NA'

In [65]:
df.isnull().sum()

country      0
alpha-2      0
alpha-3      0
continent    0
anthem       0
dtype: int64

## Data preprocessing

In [67]:
anthem_corpus = df['anthem'].tolist()

In [71]:
df.loc[df['country'] == 'Nepal']

,country,alpha-2,alpha-3,continent,anthem
114,Nepal,NP,NPL,Asia,"Woven from hundreds of flowers, we are one gar..."


In [72]:
anthem_corpus[114]

"Woven from hundreds of flowers, we are one garland that's Nepali Spread sovereign from Mechi to Mahakali. A shawl of nature's wealth unending From the blood of the braves, a nation free and non-moving. A land of knowledge, of peace, the plains, hills and mountains tall Indivisible, this beloved land of ours, our motherland Nepal. Of many races, languages, religions, and cultures of incredible sprawl This progressive nation of ours, all hail Nepal!"